<a href="https://colab.research.google.com/github/atharv-d21/titanic-mL-model/blob/main/titanic_survival_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# imports for PyTorch
import torch
from torch import nn
torch.__version__

'2.6.0+cu124'

In [35]:
# import for other tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [36]:
# importing training and test data in csv file to pandas dataframe
train_data = pd.read_csv("https://raw.githubusercontent.com/atharv-d21/titanic-mL-model/refs/heads/main/Data/train.csv", index_col = False)
test_data = pd.read_csv("https://raw.githubusercontent.com/atharv-d21/titanic-mL-model/refs/heads/main/Data/test.csv", index_col = False)

train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [37]:
test_data.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [38]:
# droping passanger's names as their names are less likely to affect survival chances
train_data.drop(columns = ['Name', 'Ticket'], inplace = True)
test_data.drop(columns = ['Name', 'Ticket'], inplace = True)

In [39]:
# replacing the ages of unknown idenviduals to mean average age for standardization
train_data['Age'] = train_data['Age'].fillna(29.7)
test_data['Age'] = test_data['Age'].fillna(29.7)

In [40]:
# replacing male and female with 0 and 1 respectively for standardization
train_data['Sex'] = train_data['Sex'].replace(['male'], 0)
train_data['Sex'] = train_data['Sex'].replace(['female'], 1)
test_data['Sex'] = test_data['Sex'].replace(['male'], 0)
test_data['Sex'] = test_data['Sex'].replace(['female'], 1)

/tmp/ipython-input-40-4272057895.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Sex'] = train_data['Sex'].replace(['female'], 1)
/tmp/ipython-input-40-4272057895.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['Sex'] = test_data['Sex'].replace(['female'], 1)


In [41]:
train_data.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,0,22.0,1,0,7.2500,NaN,S
1,2,1,1,1,38.0,1,0,71.2833,C85,C
2,3,1,3,1,26.0,0,0,7.9250,NaN,S
3,4,1,1,1,35.0,1,0,53.1000,C123,S
4,5,0,3,0,35.0,0,0,8.0500,NaN,S
5,6,0,3,0,29.7,0,0,8.4583,NaN,Q
6,7,0,1,0,54.0,0,0,51.8625,E46,S
7,8,0,3,0,2.0,3,1,21.0750,NaN,S
8,9,1,3,1,27.0,0,2,11.1333,NaN,S
9,10,1,2,1,14.0,1,0,30.0708,NaN,C


In [42]:
# filling null values for 'Embarked' with mode value
train_data['Embarked'] = train_data['Embarked'].fillna('S')
test_data['Embarked'] = test_data['Embarked'].fillna('S')

In [43]:
# replacing the null values with "UNKNOWN" for elimination of all null values from the data
train_data['Cabin'] = train_data['Cabin'].fillna('UNKNOWN')
test_data['Cabin'] = test_data['Cabin'].fillna('UNKNOWN')

In [44]:
train_data.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,0,22.0,1,0,7.2500,UNKNOWN,S
1,2,1,1,1,38.0,1,0,71.2833,C85,C
2,3,1,3,1,26.0,0,0,7.9250,UNKNOWN,S
3,4,1,1,1,35.0,1,0,53.1000,C123,S
4,5,0,3,0,35.0,0,0,8.0500,UNKNOWN,S
5,6,0,3,0,29.7,0,0,8.4583,UNKNOWN,Q
6,7,0,1,0,54.0,0,0,51.8625,E46,S
7,8,0,3,0,2.0,3,1,21.0750,UNKNOWN,S
8,9,1,3,1,27.0,0,2,11.1333,UNKNOWN,S
9,10,1,2,1,14.0,1,0,30.0708,UNKNOWN,C


I looked into a few things and found out that the First alphabet of the cabin code provides us with the information on the floor of location of the cabins which can be determining factor for the survivial of the passengers.

So, some key survival factors are :

    Sex

    Age

    Cabin

    PClass

Also, some minor factors affecting the survival rate can be :

    Parch

    SibSp

    Embarked

    Fare

In [45]:
# lets replace cabin with cabin_floor which is essentailly the first alphabet of the cabin code
train_data['CabinFloor'] = train_data['Cabin'].str[0]
test_data['CabinFloor'] = test_data['Cabin'].str[0]

In [46]:
train_data.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinFloor
0,1,0,3,0,22.0,1,0,7.2500,UNKNOWN,S,U
1,2,1,1,1,38.0,1,0,71.2833,C85,C,C
2,3,1,3,1,26.0,0,0,7.9250,UNKNOWN,S,U
3,4,1,1,1,35.0,1,0,53.1000,C123,S,C
4,5,0,3,0,35.0,0,0,8.0500,UNKNOWN,S,U
5,6,0,3,0,29.7,0,0,8.4583,UNKNOWN,Q,U
6,7,0,1,0,54.0,0,0,51.8625,E46,S,E
7,8,0,3,0,2.0,3,1,21.0750,UNKNOWN,S,U
8,9,1,3,1,27.0,0,2,11.1333,UNKNOWN,S,U
9,10,1,2,1,14.0,1,0,30.0708,UNKNOWN,C,U


In [47]:
train_data['CabinFloor'].value_counts()

,count
CabinFloor,
U,687
C,59
B,47
D,33
E,32
A,15
F,13
G,4
T,1


In [48]:
# dropping "Cabin" before one-hot encoding the data
train_data = train_data.drop(columns = ['Cabin'])
test_data = test_data.drop(columns = ['Cabin'])

In [49]:
# one-hot encoding the training data
one_hot_train_data = pd.get_dummies(train_data, dtype = float)
one_hot_train_data.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,CabinFloor_A,CabinFloor_B,CabinFloor_C,CabinFloor_D,CabinFloor_E,CabinFloor_F,CabinFloor_G,CabinFloor_T,CabinFloor_U
0,1,0,3,0,22.0,1,0,7.2500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,1,1,38.0,1,0,71.2833,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,3,1,26.0,0,0,7.9250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,1,1,35.0,1,0,53.1000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,3,0,35.0,0,0,8.0500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,6,0,3,0,29.7,0,0,8.4583,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,7,0,1,0,54.0,0,0,51.8625,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,8,0,3,0,2.0,3,1,21.0750,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,9,1,3,1,27.0,0,2,11.1333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,10,1,2,1,14.0,1,0,30.0708,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:
# one-hot encoding the testing data
one_hot_test_data = pd.get_dummies(test_data, dtype = float)
one_hot_test_data.head(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,CabinFloor_A,CabinFloor_B,CabinFloor_C,CabinFloor_D,CabinFloor_E,CabinFloor_F,CabinFloor_G,CabinFloor_U
0,892,3,0,34.5,0,0,7.8292,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,893,3,1,47.0,1,0,7.0000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,894,2,0,62.0,0,0,9.6875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,895,3,0,27.0,0,0,8.6625,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,896,3,1,22.0,1,1,12.2875,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,897,3,0,14.0,0,0,9.2250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,898,3,1,30.0,0,0,7.6292,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,899,2,0,26.0,1,1,29.0000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,900,3,1,18.0,0,0,7.2292,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,901,3,0,21.0,2,0,24.1500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [51]:
# storing all passenger IDs before dropping them as they are redundent to the result
train_pid = one_hot_train_data['PassengerId']
test_pid = one_hot_test_data['PassengerId']

# dropping the Passenger IDs
one_hot_train_data = one_hot_train_data.drop(columns = ['PassengerId'])
one_hot_test_data = one_hot_test_data.drop(columns = ['PassengerId'])

In [52]:
# spliting the data into training features and labels
train_X = one_hot_train_data.drop(columns = ['Survived', 'CabinFloor_T'])
train_y = one_hot_train_data['Survived']

test_X = one_hot_test_data

In [53]:
train_X.columns
# input feature of our model are 18 -> output label is 1.

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C',
       'Embarked_Q', 'Embarked_S', 'CabinFloor_A', 'CabinFloor_B',
       'CabinFloor_C', 'CabinFloor_D', 'CabinFloor_E', 'CabinFloor_F',
       'CabinFloor_G', 'CabinFloor_U'],
      dtype='object')

In [54]:
# converting the features and labels to tensors
numpy_train_X = train_X.to_numpy().astype(np.float32)
numpy_train_y = train_y.to_numpy().astype(np.float32)
numpy_test_X = test_X.to_numpy().astype(np.float32)

In [55]:
TRAIN_X = torch.from_numpy(numpy_train_X)
TRAIN_Y = torch.from_numpy(numpy_train_y)
TEST_X = torch.from_numpy(numpy_test_X)

In [56]:
# device agnoustic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [57]:
# lets put our tensors into gpu for faster processing
TRAIN_X = TRAIN_X.to(device)
TRAIN_Y = TRAIN_Y.to(device)
TEST_X = TEST_X.to(device)

In [58]:
# lets set the hyper parameters first
INPUT_FEATURES = 17
OUTPUT_FEATURES = 1

In [59]:
# now lets make a base line model for our problem
class survival_classification_model(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Linear(in_features = INPUT_FEATURES, out_features = 32), # 18 -> 32
        nn.ReLU(),
        nn.Linear(in_features = 32, out_features = 64),
        nn.Linear(in_features = 64, out_features = 64),
        nn.Linear(in_features = 64, out_features = 32),
        nn.Linear(in_features = 32, out_features = 16),
        nn.Linear(in_features = 16, out_features = 8),
        nn.ReLU(),
        nn.Linear(in_features = 8, out_features = OUTPUT_FEATURES)  # 8 -> 1
    )

  def forward(self, x):
    return self.layer_stack(x)

In [60]:
model_0 = survival_classification_model()
model_0.to(device)

survival_classification_model(
  (layer_stack): Sequential(
    (0): Linear(in_features=17, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=16, bias=True)
    (6): Linear(in_features=16, out_features=8, bias=True)
    (7): ReLU()
    (8): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [61]:
# make an accuracy function
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct / len(y_pred)) * 100
  return acc

In [62]:
model_params = model_0.state_dict()

In [63]:
# setting up a loss finction
loss_fn = torch.nn.BCELoss()

# setting up an optimizer
optimizer = torch.optim.SGD(params = model_0.parameters(), lr = 0.15)

In [65]:
# making a training loop for training features and labels
epochs = 1000

for epoch in range(epochs):
  model_0.train()

  # make logits
  logits = model_0(TRAIN_X) # Remove .squeeze() here, let the output be [891, 1]
  # pred_prob = torch.sigmoid(logits) # Calculate probability AFTER loss if needed for metrics
  # pred = torch.round(pred_prob) # Calculate prediction AFTER loss if needed for metrics

  # Calculate the loss
  # Apply sigmoid to logits to get probabilities between 0 and 1
  pred_prob = torch.sigmoid(logits)
  # Reshape TRAIN_Y to [891, 1] to match pred_prob dimension
  loss = loss_fn(pred_prob, TRAIN_Y.float().unsqueeze(1))

  # optimizer zero grad
  optimizer.zero_grad()

  # loss back propagation
  loss.backward()

  # optimizer step
  optimizer.step()

  model_0.eval()
  with torch.inference_mode():
    logits = model_0(TEST_X)
    pred_prob = torch.sigmoid(logits)
    pred = torch.round(pred_prob)

In [68]:
pred.shape, TEST_X.shape

(torch.Size([418, 1]), torch.Size([418, 17]))

In [69]:
pred_numpy = pred.cpu().numpy()

In [70]:
pred_df = pd.DataFrame({"PassengerId" : test_pid, "Survived" : pred_numpy.flatten()})

In [73]:
pred_df['Survived'].value_counts()

,count
Survived,
0.0,327
1.0,90


In [74]:
# export result in csv file format
pred_df.to_csv("submission.csv", index = False)